# SC32x 
## 자연어처리 (Natural Language Processing)

# Part 1 : 개념 요약

> 다음의 키워드에 대해서 **한 줄**로 간단하게 요약해주세요. (세션 노트를 참고하여도 좋습니다.)<br/>
> **Tip : 아래 문제를 먼저 수행한 후 모델 학습 등 시간이 오래 걸리는 셀이 실행되는 동안 아래 내용을 작성하면 시간을 절약할 수 있습니다.**

**N321**
- Stopwords(불용어)
- Stemming과 Lemmatization
- Bag-of-Words
- TF-IDF

- Stopwords(불용어) : 해석하고자 하는 관점에서 아무런 의미가 없는 단어
- Stemming과 Lemmatization : 단어의 원형을 찾기 위한 텍스트 정규화 기법
- Bag-of-Words : 문서를 단어의 집합으로 표현하는 방법
- TF-IDF : 단어의 상대적인 중요성을 계산하는 방법


**N322**
- Word2Vec
- fastText

- Word2Vec : 단어를 벡터로 표현하는 방법
- fastText : subword 단위의 정보를 고려하여 OOV(Out-Of-Vocabulary) 문제를 해결

**N323**
- RNN
- LSTM, GRU
- Attention

- RNN : 순환 구조를 가진 신경망으로, 시퀀스 데이터의 처리에 적합
- LSTM, GRU : 장기 의존성 문제를 해결하기 위해 제안된 모델
- Attention : RNN이나 LSTM, GRU 모델에서 중요한 입력에 집중하고 가중치를 부여하기 위해 사용되는 메커니즘

# Part 2 : Fake/Real News Dataset

한 주간 자연어처리 기법을 배우면서 여러분은 다양한 기술들을 접했습니다.<br/>
어떻게 텍스트 데이터를 다뤄야 하는지, 텍스트를 벡터화 하는 법, 문서에서 토픽을 모델하는 법 등 다양한 NLP 기법을 배웠는데요.<br/>
이번 스프린트 챌린지에선 [Fake/Real News Dataset](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset)을 사용하여 배운 것들을 복습해보는 시간을 갖겠습니다.

**주의 : 모델의 성능을 최대한 끌어올리는 것이 아닌 모델 구동에 초점을 맞춰주세요.<br/>
모든 문제를 완료한 후에도 "시간이 남았다면" 정확도를 올리는 것에 도전하시는 것을 추천드립니다.**

In [1]:
# 코드 실행 전 seed를 지정하겠습니다.
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

## 2.0 데이터셋을 불러옵니다.

- 위 캐글 링크에서 데이터셋을 받아 업로드 합니다.<br/>
(직접 업로드하게 되면 시간이 꽤 걸리므로 **drive_mount** 나 **kaggle 연동**하시는 것을 추천드립니다.)

- 'label' 열을 만들어 Fake = 1, True = 0 로 레이블링해줍니다.
- 두 파일을 합쳐 하나의 데이터프레임에 저장해 준 후 데이터를 섞어줍니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

fake = pd.read_csv("/content/drive/MyDrive/Fake Real News Dataset/Fake.csv")
true = pd.read_csv("/content/drive/MyDrive/Fake Real News Dataset/True.csv")

fake['label'],true['label'] = 1,0
df = pd.concat([fake, true], axis=0)
df = df.sample(frac=1).reset_index(drop=True)
df = df[['text', 'label']]

## 2.1 TF-IDF 를 활용하여 특정 뉴스와 유사한 뉴스 검색하기

시간상 특별한 **전처리 없이** 아래 태스크를 수행하겠습니다.

In [4]:
df

,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",1
1,WASHINGTON (Reuters) - U.S. President Donald T...,0
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,0
3,"On Monday, Donald Trump once again embarrassed...",1
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",0
...,...,...
44893,,1
44894,LONDON/TOKYO (Reuters) - British Prime Ministe...,0
44895,BERLIN (Reuters) - Chancellor Angela Merkel sa...,0
44896,Jesus f*cking Christ our President* is a moron...,1


### 2.1.1 TFidfVectorizer를 사용하여 문서-단어 행렬(Document-Term Matrix) 만들기

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english',max_features=5000)
document_term_matrix = vectorizer.fit_transform(df['text'])
document_term_matrix = pd.DataFrame(document_term_matrix.todense(), columns=vectorizer.get_feature_names_out())
document_term_matrix

,00,000,10,100,11,12,120,13,14,15,...,youtube,ypg,zealand,zero,zika,zimbabwe,zone,zones,zor,zuma
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030210,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
44894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.052385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
44895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
44896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.445167,0.125232,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.029342,0.0,0.0,0.0


### 2.1.2 KNN 알고리즘을 사용하여 유사한 문서 검색하기

- **42번 인덱스의 문서**와 가장 유사한 **5개 문서(42번 포함)의 인덱스**와 **해당 인덱스의 레이블**을 나타내주세요.
- NN 모델의 파라미터 중 `algorithm = 'kd_tree'` 로 설정합니다.

In [6]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
knn.fit(document_term_matrix)

distances, indices = knn.kneighbors([document_term_matrix.iloc[42]])

# 검색된 인덱스와 해당 인덱스의 레이블 출력
for i, index in enumerate(indices[0]):
    print(f"Index: {index}, Label: {df['label'][index]}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index: 33954, Label: 1
Index: 42, Label: 1
Index: 41, Label: 1
Index: 11, Label: 1
Index: 177, Label: 1


## 2.2 Keras Embedding을 사용하여 분류하기

### 2.2.0 데이터셋 split

- Train, Test 데이터셋으로 분리(Split)하여 주세요.

In [7]:
from sklearn.model_selection import train_test_split

# 특성 벡터와 레이블 데이터 분할
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape,X_test.shape

((33673,), (11225,))

### 2.2.1 단어 벡터의 평균을 이용하여 분류해보기

N322에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길고 시간이 오래 걸리므로 시간상 epoch 수를 **10 이하**로 하는 것을 추천드립니다.<br/>
모델 구동이 목적이므로 임베딩 차원 수를 크지 않게(50이하)로 설정해주세요.<br/>
**권장사항 : `max_len` 은 텍스트 길이 평균보다 높게 설정해주세요.**<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [8]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D,LSTM

In [9]:
print(f'학습 데이터에 있는 문서의 평균 토큰 수: {np.mean([len(sent) for sent in X_train], dtype=int)}')

학습 데이터에 있는 문서의 평균 토큰 수: 2464


In [10]:
max_len = 3000
embedding_dim = 50

token = Tokenizer()
token.fit_on_texts(X_train)

sequences = token.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
test_sequences = token.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [11]:
vocab_size = len(token.word_index) + 1
print(vocab_size)

122123


In [12]:
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(GlobalAveragePooling1D())
model1.add(Dense(1, activation='sigmoid'))

In [13]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(sequences_matrix, y_train, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
106/106 [==============================] - 31s 238ms/step - loss: 0.6873 - accuracy: 0.5235 - val_loss: 0.6808 - val_accuracy: 0.5149
Epoch 2/10
106/106 [==============================] - 24s 229ms/step - loss: 0.6672 - accuracy: 0.5814 - val_loss: 0.6508 - val_accuracy: 0.8812
Epoch 3/10
106/106 [==============================] - 21s 199ms/step - loss: 0.6269 - accuracy: 0.8426 - val_loss: 0.6004 - val_accuracy: 0.8986
Epoch 4/10
106/106 [==============================] - 22s 209ms/step - loss: 0.5696 - accuracy: 0.9230 - val_loss: 0.5386 - val_accuracy: 0.9176
Epoch 5/10
106/106 [==============================] - 19s 182ms/step - loss: 0.5065 - accuracy: 0.9355 - val_loss: 0.4775 - val_accuracy: 0.9497
Epoch 6/10
106/106 [==============================] - 20s 189ms/step - loss: 0.4471 - accuracy: 0.9348 - val_loss: 0.4219 - val_accuracy: 0.9151
Epoch 7/10
106/106 [==============================] - 16s 147ms/step - loss: 0.3954 - accuracy: 0.9422 - val_loss: 0.3758 - val_ac

In [14]:
acc1 = model1.evaluate(test_sequences_matrix,y_test)
print(acc1)

351/351 [==============================] - 1s 4ms/step - loss: 0.2807 - accuracy: 0.9424
[0.280699223279953, 0.9424498677253723]


### 2.2.2 LSTM을 사용하여 텍스트 분류 수행해보기

N323에서 했던 단어 임베딩 벡터의 평균을 사용하여 문장을 분류하는 작업을 수행해봅시다.<br/>
인스턴스마다 텍스트 길이가 길어 시간이 매우 오래 걸리므로 <br/>
**층을 최소한으로 쌓고**, epoch 수를 **3 이하**로 하는 것을 추천드립니다.<br/>

> **Tip : 모델이 학습하는 동안 2.2.3의 내용을 작성하면 시간을 절약할 수 있습니다.**


In [15]:
model2 = Sequential([
  Embedding(vocab_size, embedding_dim,input_length=max_len),
  LSTM(8, dropout=0.2, recurrent_dropout=0.2),
  Dense(1, activation='sigmoid') 
])

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(sequences_matrix, y_train, epochs=3, batch_size=256,validation_split=0.2)

Epoch 1/3
106/106 [==============================] - 1607s 15s/step - loss: 0.4597 - accuracy: 0.8915 - val_loss: 0.1977 - val_accuracy: 0.9540
Epoch 2/3
106/106 [==============================] - 1620s 15s/step - loss: 0.1299 - accuracy: 0.9727 - val_loss: 0.1065 - val_accuracy: 0.9725
Epoch 3/3
106/106 [==============================] - 1598s 15s/step - loss: 0.0705 - accuracy: 0.9869 - val_loss: 0.0782 - val_accuracy: 0.9814


In [16]:
acc2 = model2.evaluate(test_sequences_matrix,y_test)

351/351 [==============================] - 293s 836ms/step - loss: 0.0729 - accuracy: 0.9824


### 2.2.3 위에서 실행한 내용에 대해 다시 알아봅시다.

#### a) 데이터셋을 학습할 때 사용하는 `pad_sequences`  메서드에 대해 설명해주세요.<br/>어떤 기능을 하나요? 모델을 학습할 때 왜 필요한가요?

데이터의 길이를 맞춰주기 위해 사용됨.

모든 입력 데이터가 동일한 길이를 가져야 모델이 일관된 형태로 처리할 수 있습니다

#### b) 2.2.1과 2.2.2에서 사용한 각 모델의 evaluation 성능은 어떻게 나왔나요?<br/>각 모델의 장단점은 무엇이라고 생각하나요?

- KNN
  - 장점
    1. 단순하고 직관적인 알고리즘입니다. 이해하기 쉽고 구현하기 간단합니다.
    2. 학습 단계가 없으며, 모델을 훈련시키는 과정이 없습니다. 새로운 데이터에 대한 예측을 즉시 수행할 수 있습니다.
    3. 다양한 유형의 데이터에 적용할 수 있습니다.
    4. 이상치에 상대적으로 덜 민감합니다.
  - 단점
    1. 예측 속도가 느리며, 대량의 데이터에 대해 비효율적입니다. 모든 학습 데이터와의 거리를 계산해야 하기 때문입니다.
    2. 특성이 많거나 차원이 높은 데이터에는 성능이 저하될 수 있습니다.
    3. 데이터의 불균형 문제가 발생할 수 있습니다. 클래스의 분포가 불균형하면 예측 성능이 저하될 수 있습니다.

- LSTM
  - 장점
    1. 순차적인 데이터(시계열 데이터, 자연어 등)를 처리하는데 강점을 가지고 있습니다.
    2. 기존의 RNN(Recurrent Neural Network)에 비해 긴 시퀀스를 잘 학습할 수 있고, 시간적인 의존성을 잘 캡처할 수 있습니다.
    3. 학습된 모델은 새로운 시퀀스 데이터에 대해 예측을 실시간으로 수행할 수 있습니다.
  - 단점
    1. 모델 구조가 복잡하고 많은 파라미터를 가지기 때문에 학습이 상대적으로 느릴 수 있습니다.
    2. 오버피팅(Overfitting) 문제가 발생할 수 있습니다. 적절한 정규화와 조절이 필요합니다.
    3. 장기적인 의존성을 학습하기 위해 긴 시퀀스 데이터가 필요할 수 있습니다.

#### c) 종래의 RNN(Recurrent Neural Networks) 대신 LSTM(Long-Short Term Memory)을 사용하는 이유는 무엇인가요?<br/>(i.e. RNN에 비해 LSTM의 좋은 점을 설명해주세요.)

메모리 셀과 게이트 메커니즘을 사용하여 장기 의존성을 효과적으로 학습할 수 있습니다.

#### d) LSTM이나 RNN을 사용하는 예시를 **3개**이상 제시하고 해당되는 경우에 왜 LSTM이나 RNN을 사용하는 것 적절한지 간단하게 설명해주세요.

1. 기계 번역: 입력 문장과 출력 문장의 시퀀스 간의 번역 작업에서 LSTM이나 RNN을 사용할 수 있습니다. 문장의 단어 순서와 문맥을 고려해야 하기 때문에 순차적인 정보를 처리할 수 있는 RNN 계열의 모델이 적합합니다.

2. 감성 분석: 문장이나 문서의 감성(긍정 또는 부정)을 분류하는 작업에서 LSTM이나 RNN을 사용할 수 있습니다. 문장 내의 단어 순서와 문맥이 감성을 결정하는데 중요한 역할을 하기 때문에 RNN 계열의 모델을 활용할 수 있습니다.

3. 텍스트 생성: 주어진 문맥에서 새로운 텍스트를 생성하는 작업에서 LSTM이나 RNN을 사용할 수 있습니다.

#### e) 이외에 N324 에서 배운 자연어처리 모델과 관련된 키워드를 3개 이상 적어주세요. <br/> (해당 키워드에 대한 설명은 옵션입니다.)

Transformer,BERT,GPT

# Advanced Goals: 3점을 획득하기 위해선 아래의 조건 중 하나 이상을 만족해야합니다
 
- 2.1 에서 TF-IDF(`TfidfVectorizer`)가 아닌 방법을 사용하여 유사도 검색을 수행해보세요.<br/>
TF-IDF와 해당 방법의 차이를 설명해주세요. 
- 2.2 에서 사용한 방법을 재사용하되 하이퍼 파라미터를 조정하거나 모델 구조를 변경하여 성능을 올려봅시다.<br/>**(주의 : GridSearch, RandomSearch 등의 방법을 사용하여도 좋으나 시간이 오래 걸리므로 범위를 잘 선택해야 합니다.)**

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english',max_features=5000)
document_term_matrix = vectorizer.fit_transform(df['text'])
document_term_matrix

<44898x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 5462202 stored elements in Compressed Sparse Row format>

In [18]:
document_term_matrix = pd.DataFrame(document_term_matrix.todense(), columns=vectorizer.get_feature_names_out())
document_term_matrix

,00,000,10,100,11,12,120,13,14,15,...,youtube,ypg,zealand,zero,zika,zimbabwe,zone,zones,zor,zuma
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44894,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
44895,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44896,0,0,0,0,0,0,0,18,5,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
knn.fit(document_term_matrix)

distances, indices = knn.kneighbors([document_term_matrix.iloc[42]])

# 검색된 인덱스와 해당 인덱스의 레이블 출력
for i, index in enumerate(indices[0]):
    print(f"Index: {index}, Label: {df['label'][index]}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Index: 33954, Label: 1
Index: 42, Label: 1
Index: 9838, Label: 1
Index: 28385, Label: 1
Index: 35697, Label: 1


In [20]:
import tensorflow as tf
import IPython

!pip install -U keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 10.5 MB/s eta 0:00:00


In [40]:
pip install --upgrade keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(Embedding(vocab_size, embedding_dim,input_length=max_len)),
  hp_units = hp.Choice('units', values =[16,32,64])
  model.add(LSTM(units=hp_units)),
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = tf.keras.losses.binary_crossentropy,
                metrics = ['accuracy'])
  return model

In [38]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', # 학습 진행 지표를 문자열로 받습니다.
                     max_epochs = 10, # 학습을 진행할 최대 epoch 수입니다.
                     factor = 3, # iteration, epoch 등을 감소시킬 계수이며 default=3입니다.
                     directory = 'my_dir', # 튜닝 결과를 저장할 directory를 설정합니다.
                     project_name = 'intro_to_kt') # 저장하는 튜닝 결과의 이름입니다.

In [39]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(sequences_matrix, y_train, epochs = 10, validation_data = (test_sequences_matrix, y_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0] # 반환할 최고의 조합을 선언합니다. num_trials = 1은 개체 수를 의미합니다.

print(f"""
하이퍼 파라미터 검색이 완료되었습니다. 
최적화된 dropout 비율은 {best_hps.get('units')} 입니다.
최적의 학습 속도는 {best_hps.get('learning_rate')} 입니다.
""")


Search: Running Trial #9

Value             |Best Value So Far |Hyperparameter
16                |16                |units
0.01              |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1053/1053 [==============================] - 392s 370ms/step - loss: 0.0960 - accuracy: 0.9676 - val_loss: 0.0069 - val_accuracy: 0.9982
Epoch 2/2
 302/1053 [=======>......................] - ETA: 4:44 - loss: 0.0137 - accuracy: 0.9963

충분히 좋은 성능이 보이고 있지만.. 런타임이 끊겼습니다 ㅠ

In [ ]:
model = tuner.hypermodel.build(best_hps) # 최고 성능의 조합으로 모델을 다시 구축합니다.

model.summary()

In [ ]:
model.evaluate(test_sequences_matrix,y_test)